# End-to-End Demo (BB)

Run a small sweep over physical error rates and report logical failure rates.


In [1]:
import numpy as np
import stim

from quits.circuit import check_overlapping_CX
from quits import ErrorModel, CircuitBuildOptions
from quits.decoder import (
    sliding_window_bposd_circuit_mem,
    sliding_window_bposd_phenom_mem,
    sliding_window_bplsd_phenom_mem,
)
from quits.qldpc_code import BbCode
from quits.simulation import get_stim_mem_result


In [2]:
# Common parameters
seed = 1
p = 1e-3
num_rounds = 15
num_trials = 1000
W, F = 5, 3
max_iter, osd_order = 10, 1


In [3]:
# Build a small BB code
code = BbCode(
    l=15,
    m=3,
    A_x_pows=[9],
    A_y_pows=[1, 2],
    B_x_pows=[2, 7],
    B_y_pows=[0],
)
report = code.verify_css_logicals()
print("verify_css_logicals", report)


verify_css_logicals {'css_condition': True, 'lz_commutes_with_X': True, 'lx_commutes_with_Z': True, 'rank_hz': 41, 'rank_hx': 41, 'rank_lz': 8, 'rank_lx': 8, 'k_expected': 8, 'lz_independent_mod_Z_stabilizers': True, 'lx_independent_mod_X_stabilizers': True, 'rank_hz_plus_lz': 49, 'rank_hx_plus_lx': 49, 'dim_ker_hz': 49, 'dim_ker_hx': 49, 'hx_plus_lx_spans_ker_hz': True, 'hz_plus_lz_spans_ker_hx': True, 'same_logicals_ZX_anticommute': True, 'different_logicals_ZX_commute': True, 'all_tests_passed': True}


### Outputs/metrics
- `pL`: logical error rate per experiment (any logical flip)
- `LFR`: logical failure rate per round, `1 - (1 - pL)^(1/num_rounds)`


In [4]:
# Small sweep over p
ps = [5e-4, 1e-3, 2e-3]
results = []

for p in ps:
    circuit = code.build_circuit(
        error_model=ErrorModel(p, p, p, p),
        num_rounds=num_rounds,
        basis="Z",
        circuit_build_options=CircuitBuildOptions(),
    )
    
    detection_events, observable_flips = get_stim_mem_result(
        circuit, num_trials, seed=seed
    )

    logical_pred = sliding_window_bposd_circuit_mem(
        detection_events,
        circuit,
        code.hz,
        code.lz,
        W,
        F,
        max_iter=max_iter,
        osd_order=osd_order,
        tqdm_on=True,
    )

    pL = np.mean((observable_flips - logical_pred).any(axis=1))
    lfr = 1 - (1 - pL) ** (1 / num_rounds)
    results.append((p, float(lfr)))
    
print(results)


100%|██████████| 1000/1000 [00:40<00:00, 24.61it/s]

[(0.0005, 0.0), (0.001, 6.669779784185259e-05), (0.002, 0.00013345793840846554)]
